# 进程与线程区别
* 同一个进程中的线程共享同一内存空间，但是进程之间是独立的。
* 同一个进程中的所有线程的数据是共享的（进程通讯），进程之间的数据是独立的。
* 对主线程的修改可能会影响其他线程的行为，但是父进程的修改（除了删除以外）不会影响其他子进程。
* 线程是一个上下文的执行指令，而进程则是与运算相关的一簇资源。
* 同一个进程的线程之间可以直接通信，但是进程之间的交流需要借助中间代理来实现。
* 创建新的线程很容易，但是创建新的进程需要对父进程做一次复制。
* 一个线程可以操作同一进程的其他线程，但是进程只能操作其子进程。
* 线程启动速度快，进程启动速度慢（但是两者运行速度没有可比性）。




## Thread类
### 1.普通创建方式

In [2]:
import threading
import time


def run(n):
    print(f"task:{n}")
    time.sleep(1)
    print("2s")
    time.sleep(1)
    print("1s")
    time.sleep(1)
    print("0s")
    time.sleep(1)


t1 = threading.Thread(target=run, args=("t1",))
t2 = threading.Thread(target=run, args=("t2",))
t1.start()
t2.start()
t1.join()
t2.join()

task:t1task:t2

2s
2s
1s
1s
0s
0s


### 继承threading.Thread 类自定义线程类

In [3]:
import threading
import time


class MyThread(threading.Thread):
    def __init__(self, n):
        super(MyThread, self).__init__()  # 重构run函数必须要写
        self.n = n

    def run(n):
        print(f"task:{n}")
        time.sleep(1)
        print("2s")
        time.sleep(1)
        print("1s")
        time.sleep(1)
        print("0s")
        time.sleep(1)


if __name__ == "__main__":
    #     t1 = threading.Thread(target=run,args=('t1',))
    #     t2 = threading.Thread(target=run,args=('t2',))

    t1 = MyThread("t1")
    t2 = MyThread("t2")

    t1.start()
    t2.start()
    t1.join()
    t2.join()

task:<MyThread(Thread-6, started 123145370406912)>
task:<MyThread(Thread-7, started 123145375662080)>
2s2s

1s1s

0s0s



### 计算自线程的执行时间

In [ ]:
join()  # 等此线程执行完后，再执行其他线程或主线程
threading.current_thread()  # 输出当前线程

In [6]:
import threading
import time


def run(n):
    print("task", n, threading.current_thread())  # 输出当前的线程
    time.sleep(1)
    print("3s")
    time.sleep(1)
    print("2s")
    time.sleep(1)
    print("1s")


strat_time = time.time()

t_obj = []  # 定义列表用于存放子线程实例

for i in range(3):
    t = threading.Thread(target=run, args=("t-%s" % i,))
    t.start()
    t_obj.append(t)

"""
由主线程生成的三个子线程
task t-0 <Thread(Thread-8, started 123145370406912)>
task t-1 <Thread(Thread-9, started 123145375662080)>
task t-2 <Thread(Thread-10, started 123145380917248)>
"""

for tmp in t_obj:
    t.join()  # 为每个子线程添加join之后，主线程就会等这些子线程执行完之后再执行。

print("cost:", time.time() - strat_time)  # 主线程

# print(threading.current_thread())  # 输出当前线程


tasktask t-0 <Thread(Thread-14, started 123145370406912)>
 t-1 <Thread(Thread-15, started 123145375662080)>
task t-2 <Thread(Thread-16, started 123145380917248)>
3s3s3s


2s2s2s


1s1s

cost:1s
 3.0132899284362793


### 统计当前活跃的线程数

In [9]:
import threading
import time

def fun(n):
    print(f'任务:{n}')
    time.sleep(1)
t = []
for _ in range(3):
    t.append(threading.Thread(target=fun,args=('t-%s'%_,)))
    t[_].start()
for _ in range(3):
    t[_].join()
time.sleep(5.5)
print(threading.active_count()) #输出当前活跃的线程数

任务:t-0
任务:t-1
任务:t-2
5


## 事件(Event类)

python线程的事件用于主线程控制其他线程的执行，事件是一个简单的线程同步对象，其主要提供以下几个方法：

* 方法	注释
* clear	将flag设置为“False”
* set	将flag设置为“True”
* is_set	判断是否设置了flag
* wait	会一直监听flag，如果没有检测到flag就一直处于阻塞状态
* 事件处理的机制：全局定义了一个“Flag”，当flag值为“False”，那么event.wait()就会阻塞，当flag值为“True”，那么event.wait()便不再阻塞。

In [10]:
#利用Event类模拟红绿灯
import threading
import time

event = threading.Event()


def lighter():
    count = 0
    event.set()     #初始值为绿灯,flag设为True
    while True:
        if 5 < count <=10 :
            event.clear()  # 红灯，清除标志位
            print("现在是红灯")
        elif count > 10:
            event.set()  # 绿灯，设置标志位
            count = 0
        else:
            print("现在是绿灯")

        time.sleep(1)
        count += 1

def car(name):
    while True:
        if event.is_set():      #判断是否设置了标志位
            print(f"{[name]} 开动了...")
            time.sleep(1)
        else:
            print(f'{[name]} 看到红灯,等待中...')
            event.wait() #监听flag直到为True
            print(f"{[name]} 绿灯，请通行...")

light = threading.Thread(target=lighter,)
light.start()

car = threading.Thread(target=car,args=("MINI",))
car.start()

现在是绿灯['MINI'] 开动了...

['MINI'] 开动了...现在是绿灯

['MINI'] 开动了...现在是绿灯

['MINI'] 开动了...现在是绿灯

['MINI'] 开动了...
现在是绿灯
['MINI'] 开动了...现在是绿灯

现在是红灯['MINI'] 看到红灯,等待中...

现在是红灯
现在是红灯
现在是红灯
现在是红灯
['MINI'] 绿灯，请通行...
['MINI'] 开动了...
现在是绿灯['MINI'] 开动了...

['MINI'] 开动了...
现在是绿灯
['MINI'] 开动了...现在是绿灯

现在是绿灯['MINI'] 开动了...

现在是绿灯['MINI'] 开动了...

['MINI'] 开动了...现在是红灯

['MINI'] 看到红灯,等待中...现在是红灯

现在是红灯
现在是红灯
现在是红灯
['MINI'] 绿灯，请通行...
['MINI'] 开动了...
现在是绿灯['MINI'] 开动了...

现在是绿灯['MINI'] 开动了...

现在是绿灯['MINI'] 开动了...

['MINI'] 开动了...现在是绿灯

现在是绿灯['MINI'] 开动了...



In [1]:
from  multiprocessing import Process,Pool
import time
 
def Foo(i):
    time.sleep(2)
    return i+100
 
def Bar(arg):
    print('-->exec done:',arg)
 
pool = Pool(5)  #允许进程池同时放入5个进程
 
for i in range(10):
    pool.apply_async(func=Foo, args=(i,),callback=Bar)  
    #func子进程执行完后，才会执行callback，否则callback不执行（而且callback是由父进程来执行了）
    #pool.apply(func=Foo, args=(i,))
 
print('end')
pool.close()
pool.join() #主进程等待所有子进程执行完毕。必须在close()或terminate()之后。

end
-->exec done: 100
-->exec done: 101
-->exec done: 102
-->exec done: 103
-->exec done: 104
-->exec done: 106
-->exec done: 105
-->exec done: 107
-->exec done: 108
-->exec done: 109


## 协程

线程和进程的操作是由程序触发系统接口，最后的执行者是系统，它本质上是操作系统提供的功能。而协程的操作则是程序员指定的，在python中通过yield，人为的实现并发处理。

协程存在的意义：对于多线程应用，CPU通过切片的方式来切换线程间的执行，线程切换时需要耗时。协程，则只使用一个线程，分解一个线程成为多个“微线程”，在一个线程中规定某个代码块的执行顺序。

协程的适用场景：当程序中存在大量不需要CPU的操作时（IO）。
常用第三方模块gevent和greenlet。（本质上，gevent是对greenlet的高级封装，因此一般用它就行，这是一个相当高效的模块。）

In [2]:
from greenlet import greenlet

def t1():
    print('t1')
    gr2.switch()
    print('t1.gr2.switch()')
    gr2.switch()

def t2():
    print('t2')
    gr1.switch()
    print('t2.gr1.switch()')

gr1 = greenlet(t1)
gr2 = greenlet(t2)
gr1.switch()

t1
t2
t1.gr2.switch()
t2.gr1.switch()


In [ ]:
# from gevent import monkey; monkey.patch_all()
import gevent
import requests

def f(url):
    print('GET: %s' % url)
    resp = requests.get(url)
    data = resp.text
    print('%d bytes received from %s.' % (len(data), url))

gevent.joinall([
        gevent.spawn(f, 'https://www.python.org/'),
        gevent.spawn(f, 'https://www.yahoo.com/'),
        gevent.spawn(f, 'https://github.com/'),
])
